# MNIST Dataset Analysis: PCA + K-Means Clustering

This notebook performs a comprehensive analysis of the MNIST dataset using:
- **Principal Component Analysis (PCA)** for dimensionality reduction
- **K-Means Clustering** for unsupervised classification

We'll explore how different numbers of principal components affect clustering performance.

## 1. Imports and Setup

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Machine Learning libraries
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import (
    silhouette_score,
    adjusted_rand_score,
    normalized_mutual_info_score,
    confusion_matrix
)

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Set matplotlib style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Create results directory
results_dir = Path('results')
results_dir.mkdir(exist_ok=True)

print("✓ All libraries imported successfully")
print(f"✓ Results directory created: {results_dir}")

## 2. Load and Prepare MNIST Dataset

We'll load a subset of 10,000 samples from the MNIST dataset to make computations manageable while still having enough data for meaningful analysis.

In [ ]:
# Load MNIST dataset
print("Loading MNIST dataset...")
mnist = fetch_openml('mnist_784', version=1, parser='auto')
X, y = mnist.data, mnist.target

# Convert to numpy arrays
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.int32)

# Select random subset of 10,000 samples
n_samples = 10000
np.random.seed(RANDOM_STATE)
indices = np.random.choice(len(X), n_samples, replace=False)
X_subset = X[indices]
y_subset = y[indices]

print(f"\n✓ Dataset loaded successfully")
print(f"  - Total samples: {len(X_subset):,}")
print(f"  - Features per sample: {X_subset.shape[1]:,}")
print(f"  - Classes: {np.unique(y_subset)}")
print(f"  - Data shape: {X_subset.shape}")

## 3. Data Normalization and Standardization

We'll normalize pixel values to [0, 1] range and then standardize for PCA.

In [ ]:
# Normalize to [0, 1] range
X_normalized = X_subset / 255.0

# Standardize the data (mean=0, std=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_normalized)

print("✓ Data normalized and standardized")
print(f"  - Mean: {X_scaled.mean():.6f}")
print(f"  - Std: {X_scaled.std():.6f}")
print(f"  - Min: {X_scaled.min():.6f}")
print(f"  - Max: {X_scaled.max():.6f}")

## 4. PCA Analysis - Variance Explained

Let's first analyze how many components we need to capture most of the variance in the data.

In [ ]:
# Fit PCA with all components to analyze variance
print("Performing PCA analysis...")
pca_full = PCA(random_state=RANDOM_STATE)
pca_full.fit(X_scaled)

# Calculate cumulative variance explained
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)

# Find number of components for different variance thresholds
n_components_80 = np.argmax(cumulative_variance >= 0.80) + 1
n_components_90 = np.argmax(cumulative_variance >= 0.90) + 1
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1

print(f"\n✓ PCA analysis complete")
print(f"  - Components for 80% variance: {n_components_80}")
print(f"  - Components for 90% variance: {n_components_90}")
print(f"  - Components for 95% variance: {n_components_95}")

In [ ]:
# Visualization 1: Variance Explained
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Individual variance explained
ax1.bar(range(1, 51), pca_full.explained_variance_ratio_[:50], alpha=0.7, color='steelblue')
ax1.set_xlabel('Principal Component', fontsize=12)
ax1.set_ylabel('Variance Explained Ratio', fontsize=12)
ax1.set_title('Individual Variance Explained by First 50 Components', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Cumulative variance explained
ax2.plot(range(1, 101), cumulative_variance[:100], linewidth=2.5, color='darkblue')
ax2.axhline(y=0.80, color='green', linestyle='--', linewidth=2, label=f'80% ({n_components_80} components)')
ax2.axhline(y=0.90, color='orange', linestyle='--', linewidth=2, label=f'90% ({n_components_90} components)')
ax2.axhline(y=0.95, color='red', linestyle='--', linewidth=2, label=f'95% ({n_components_95} components)')
ax2.set_xlabel('Number of Components', fontsize=12)
ax2.set_ylabel('Cumulative Variance Explained', fontsize=12)
ax2.set_title('Cumulative Variance Explained', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(results_dir / 'variance_explained.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: variance_explained.png")

## 5. Apply PCA with Different Numbers of Components

We'll apply PCA with 2, 10, and 30 components to compare their effectiveness.

In [ ]:
# Apply PCA with different numbers of components
n_components_list = [2, 10, 30]
pca_results = {}

for n_comp in n_components_list:
    print(f"\nApplying PCA with {n_comp} components...")
    pca = PCA(n_components=n_comp, random_state=RANDOM_STATE)
    X_pca = pca.fit_transform(X_scaled)
    variance_retained = pca.explained_variance_ratio_.sum()
    
    pca_results[n_comp] = {
        'pca': pca,
        'X_transformed': X_pca,
        'variance_retained': variance_retained
    }
    
    print(f"  ✓ Variance retained: {variance_retained:.4f} ({variance_retained*100:.2f}%)")
    print(f"  ✓ Transformed shape: {X_pca.shape}")

## 6. K-Means Clustering

Apply K-Means clustering (k=10) to each PCA-transformed dataset.

In [ ]:
# Apply K-Means clustering for each PCA configuration
n_clusters = 10
clustering_results = {}

for n_comp in n_components_list:
    print(f"\nApplying K-Means (k={n_clusters}) on {n_comp} components...")
    X_pca = pca_results[n_comp]['X_transformed']
    
    # Fit K-Means
    kmeans = KMeans(n_clusters=n_clusters, random_state=RANDOM_STATE, n_init=10, max_iter=300)
    labels_pred = kmeans.fit_predict(X_pca)
    
    # Calculate metrics
    silhouette = silhouette_score(X_pca, labels_pred)
    ari = adjusted_rand_score(y_subset, labels_pred)
    nmi = normalized_mutual_info_score(y_subset, labels_pred)
    
    clustering_results[n_comp] = {
        'kmeans': kmeans,
        'labels': labels_pred,
        'silhouette': silhouette,
        'ari': ari,
        'nmi': nmi
    }
    
    print(f"  ✓ Silhouette Score: {silhouette:.4f}")
    print(f"  ✓ Adjusted Rand Index: {ari:.4f}")
    print(f"  ✓ Normalized Mutual Information: {nmi:.4f}")

## 7. Results Summary

Let's create a comprehensive table with all metrics.

In [ ]:
# Create results summary table
results_summary = []

for n_comp in n_components_list:
    results_summary.append({
        'Components': n_comp,
        'Variance Retained': f"{pca_results[n_comp]['variance_retained']:.4f}",
        'Silhouette Score': f"{clustering_results[n_comp]['silhouette']:.4f}",
        'ARI': f"{clustering_results[n_comp]['ari']:.4f}",
        'NMI': f"{clustering_results[n_comp]['nmi']:.4f}"
    })

results_df = pd.DataFrame(results_summary)
print("\n" + "="*70)
print("CLUSTERING PERFORMANCE SUMMARY")
print("="*70)
print(results_df.to_string(index=False))
print("="*70)

## 8. Visualization: 2D Cluster Plots

Visualize the clustering results using the 2-component PCA.

In [ ]:
# Visualization 2: 2D Clusters - Predicted Labels
X_2d = pca_results[2]['X_transformed']
labels_2d = clustering_results[2]['labels']

plt.figure(figsize=(12, 10))
scatter = plt.scatter(X_2d[:, 0], X_2d[:, 1], c=labels_2d, cmap='tab10', 
                     alpha=0.6, s=30, edgecolors='k', linewidth=0.5)
plt.colorbar(scatter, label='Cluster')
plt.xlabel('First Principal Component', fontsize=12)
plt.ylabel('Second Principal Component', fontsize=12)
plt.title('K-Means Clustering Results (2 Components) - Predicted Labels', 
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(results_dir / 'clusters_2d_predicted.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: clusters_2d_predicted.png")

In [ ]:
# Visualization 3: 2D Clusters - True Labels
plt.figure(figsize=(12, 10))
scatter = plt.scatter(X_2d[:, 0], X_2d[:, 1], c=y_subset, cmap='tab10', 
                     alpha=0.6, s=30, edgecolors='k', linewidth=0.5)
plt.colorbar(scatter, label='Digit')
plt.xlabel('First Principal Component', fontsize=12)
plt.ylabel('Second Principal Component', fontsize=12)
plt.title('K-Means Clustering Results (2 Components) - True Labels', 
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(results_dir / 'clusters_2d_true.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: clusters_2d_true.png")

## 9. Confusion Matrices

Compare predicted clusters with true labels for each PCA configuration.

In [ ]:
# Visualization 4: Confusion Matrices
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

for idx, n_comp in enumerate(n_components_list):
    labels_pred = clustering_results[n_comp]['labels']
    cm = confusion_matrix(y_subset, labels_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx], 
                cbar_kws={'label': 'Count'}, square=True)
    axes[idx].set_xlabel('Predicted Cluster', fontsize=11)
    axes[idx].set_ylabel('True Digit', fontsize=11)
    axes[idx].set_title(f'Confusion Matrix\n({n_comp} Components)', 
                        fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig(results_dir / 'confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: confusion_matrices.png")

## 10. Metrics Comparison

Visual comparison of all performance metrics across different PCA configurations.

In [ ]:
# Visualization 5: Metrics Comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Prepare data
components = n_components_list
silhouette_scores = [clustering_results[n]['silhouette'] for n in components]
ari_scores = [clustering_results[n]['ari'] for n in components]
nmi_scores = [clustering_results[n]['nmi'] for n in components]
variance_retained = [pca_results[n]['variance_retained'] for n in components]

# Silhouette Score
axes[0, 0].plot(components, silhouette_scores, marker='o', linewidth=2.5, 
                markersize=10, color='steelblue')
axes[0, 0].set_xlabel('Number of Components', fontsize=11)
axes[0, 0].set_ylabel('Silhouette Score', fontsize=11)
axes[0, 0].set_title('Silhouette Score vs Components', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_xticks(components)

# Adjusted Rand Index
axes[0, 1].plot(components, ari_scores, marker='s', linewidth=2.5, 
                markersize=10, color='forestgreen')
axes[0, 1].set_xlabel('Number of Components', fontsize=11)
axes[0, 1].set_ylabel('Adjusted Rand Index', fontsize=11)
axes[0, 1].set_title('ARI vs Components', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_xticks(components)

# Normalized Mutual Information
axes[1, 0].plot(components, nmi_scores, marker='^', linewidth=2.5, 
                markersize=10, color='darkorange')
axes[1, 0].set_xlabel('Number of Components', fontsize=11)
axes[1, 0].set_ylabel('Normalized Mutual Information', fontsize=11)
axes[1, 0].set_title('NMI vs Components', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_xticks(components)

# Combined metrics
x = np.arange(len(components))
width = 0.25
axes[1, 1].bar(x - width, silhouette_scores, width, label='Silhouette', alpha=0.8)
axes[1, 1].bar(x, ari_scores, width, label='ARI', alpha=0.8)
axes[1, 1].bar(x + width, nmi_scores, width, label='NMI', alpha=0.8)
axes[1, 1].set_xlabel('Number of Components', fontsize=11)
axes[1, 1].set_ylabel('Score', fontsize=11)
axes[1, 1].set_title('All Metrics Comparison', fontsize=12, fontweight='bold')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(components)
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(results_dir / 'metrics_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: metrics_comparison.png")

## 11. Sample Digits from Each Cluster

Visualize sample digits from each cluster to understand what the algorithm learned.

In [ ]:
# Visualization 6: Sample digits from each cluster (30 components)
labels_30 = clustering_results[30]['labels']
n_samples_per_cluster = 10

fig, axes = plt.subplots(n_clusters, n_samples_per_cluster, figsize=(20, 20))
fig.suptitle('Sample Digits from Each Cluster (30 Components)', 
             fontsize=16, fontweight='bold', y=0.995)

for cluster in range(n_clusters):
    cluster_indices = np.where(labels_30 == cluster)[0]
    sample_indices = np.random.choice(cluster_indices, 
                                     min(n_samples_per_cluster, len(cluster_indices)), 
                                     replace=False)
    
    for i, idx in enumerate(sample_indices):
        axes[cluster, i].imshow(X_subset[idx].reshape(28, 28), cmap='gray')
        axes[cluster, i].axis('off')
        if i == 0:
            axes[cluster, i].set_title(f'Cluster {cluster}', 
                                       fontsize=11, fontweight='bold', loc='left')
        # Show true label
        axes[cluster, i].text(0.5, -0.1, f'True: {y_subset[idx]}', 
                             transform=axes[cluster, i].transAxes,
                             ha='center', fontsize=9, color='blue')

plt.tight_layout()
plt.savefig(results_dir / 'cluster_samples.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: cluster_samples.png")

## 12. Elbow Method Analysis

Determine the optimal number of clusters using the elbow method.

In [ ]:
# Visualization 7: Elbow Method
print("Performing elbow method analysis...")

# Use 30 components for this analysis
X_30 = pca_results[30]['X_transformed']
k_range = range(2, 21)
inertias = []
silhouette_scores_elbow = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    kmeans.fit(X_30)
    inertias.append(kmeans.inertia_)
    silhouette_scores_elbow.append(silhouette_score(X_30, kmeans.labels_))
    print(f"  k={k:2d}: Inertia={kmeans.inertia_:.2f}, Silhouette={silhouette_scores_elbow[-1]:.4f}")

# Plot elbow method results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Inertia plot
ax1.plot(k_range, inertias, marker='o', linewidth=2.5, markersize=8, color='darkblue')
ax1.axvline(x=10, color='red', linestyle='--', linewidth=2, label='k=10 (MNIST digits)')
ax1.set_xlabel('Number of Clusters (k)', fontsize=12)
ax1.set_ylabel('Inertia (Within-Cluster Sum of Squares)', fontsize=12)
ax1.set_title('Elbow Method - Inertia', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Silhouette score plot
ax2.plot(k_range, silhouette_scores_elbow, marker='s', linewidth=2.5, 
         markersize=8, color='darkgreen')
ax2.axvline(x=10, color='red', linestyle='--', linewidth=2, label='k=10 (MNIST digits)')
ax2.set_xlabel('Number of Clusters (k)', fontsize=12)
ax2.set_ylabel('Silhouette Score', fontsize=12)
ax2.set_title('Elbow Method - Silhouette Score', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(results_dir / 'elbow_method.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Saved: elbow_method.png")

## 13. Analysis and Interpretation

### Key Findings:

#### 1. **Variance Retention**
- 2 components retain only a small fraction of variance but provide good visualization
- 10 components capture significantly more variance
- 30 components retain most of the important information

#### 2. **Clustering Performance**
- **Silhouette Score**: Measures how well-separated the clusters are
  - Higher is better (range: -1 to 1)
  - Generally improves with more components as clusters become more distinct
  
- **Adjusted Rand Index (ARI)**: Measures similarity between predicted and true labels
  - Range: -1 to 1 (1 = perfect match)
  - Shows how well K-Means recovers the true digit structure
  
- **Normalized Mutual Information (NMI)**: Measures information shared between clusterings
  - Range: 0 to 1 (1 = perfect agreement)
  - Indicates the quality of unsupervised learning

#### 3. **Dimensionality Trade-offs**
- **2 Components**: Best for visualization, but loses too much information for accurate clustering
- **10 Components**: Good balance between dimensionality reduction and clustering performance
- **30 Components**: Best clustering performance, retains most discriminative features

#### 4. **Elbow Method Insights**
- The elbow method helps identify the optimal number of clusters
- For MNIST, k=10 is natural (10 digits), but the elbow might suggest different values
- This demonstrates the difference between unsupervised and supervised perspectives

### Conclusions:

1. **PCA is effective** for dimensionality reduction in high-dimensional image data
2. **More components generally improve clustering** but with diminishing returns
3. **K-Means can discover digit structure** without labels (unsupervised learning)
4. **There's a trade-off** between computational efficiency and clustering quality
5. The **confusion matrices** reveal which digits are commonly confused by the algorithm

### Practical Implications:

- For **visualization**: Use 2-3 components
- For **clustering**: Use 20-50 components depending on computational resources
- For **classification**: Consider supervised methods or more components
- The **choice of k** and number of components depends on the specific application and requirements

## 14. Final Summary

All analysis complete! Generated visualizations saved to the `results/` folder.

In [ ]:
# Print final summary
print("\n" + "="*70)
print("ANALYSIS COMPLETE")
print("="*70)
print("\nGenerated Visualizations:")
print("  1. ✓ variance_explained.png - PCA variance analysis")
print("  2. ✓ clusters_2d_predicted.png - 2D clustering with predicted labels")
print("  3. ✓ clusters_2d_true.png - 2D clustering with true labels")
print("  4. ✓ confusion_matrices.png - Confusion matrices for all configurations")
print("  5. ✓ metrics_comparison.png - Performance metrics comparison")
print("  6. ✓ cluster_samples.png - Sample digits from each cluster")
print("  7. ✓ elbow_method.png - Optimal cluster number analysis")
print("\nAll files saved to: results/")
print("="*70)

# Display final metrics table
print("\nFinal Performance Metrics:")
print(results_df.to_string(index=False))
print("\n" + "="*70)